In [1]:
import re
import networkx as nx
from networkx.readwrite import json_graph
from fourlang.text_to_4lang import TextTo4lang
from graphviz import Source
from scripts.parse_data import read
from scripts.similarity import Similarity
from tqdm import tqdm

In [2]:
data_frame = read("en", graded=False)
similarity = Similarity()

### Simple monolingual dictionary based baseline based on bag-of-words

In [ ]:
preds = []

for i in tqdm(range(len(data_frame))):
    index = i
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    score = data_frame.score[index]
    premise_definition = text_to_4lang.get_definition(premise) #legyen-e expand
    hypothesis_definition = text_to_4lang.get_definition(hypothesis)
    
    if pred > 0:
        preds.append(1)
    else:
        preds.append(0)

### Simple monolingual dictionary based baseline using 4lang expand

In [4]:
text_to_4lang = TextTo4lang(lang="en")

In [15]:
preds = []

for i in tqdm(range(len(data_frame))):
    index = i
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    score = data_frame.score[index]
    graph_premise = text_to_4lang.process_text(premise, True) #legyen-e expand
    graph_hypothesis = text_to_4lang.process_text(hypothesis, True)
    dot_graph_premise = graph_premise.to_dot()
    dot_graph_hypothesis = graph_hypothesis.to_dot()
    pred = similarity.asim_jac_edges(graph_hypothesis, graph_premise)
    if pred > 0:
        preds.append(1)
    else:
        preds.append(0)

100%|██████████| 316/316 [04:55<00:00,  1.28it/s]


In [18]:
with open("result_binary", "w+") as f:
    for i,pred in enumerate(preds):
        premise = data_frame.premise[i]
        hypothesis = data_frame.hypothesis[i]
        f.write(premise + " " + hypothesis + " " + str(pred) + "\n")

In [37]:
g = text_to_4lang.process_text("enemy", expand=True)

In [38]:
g.get_nodes()

['enemy', 'relate', 'of']

In [5]:
text_to_4lang.get_definition("dog")

'(from "dog and bone") phone,Phone or mobile phone.'